In [1]:
# Group Members
# Akinfemi Akin-Aluko
# Satbir Gill
# Chandana Prakash

#Notation
#Xij : unit cost of transportation of customer i to dc j

from gurobipy import *
import numpy as np
import pandas as pd

In [2]:
try:

    #Create a model
    m = Model("Project")

     
    #Data (Provided)
    utc_values = np.array(pd.read_csv('~/NetworkDesign/dc20_cust2000_inst1/unit_tran_cost.txt', header=None))
    dc_cost_values = np.array(pd.read_csv('~/NetworkDesign/dc20_cust2000_inst1/dc_cost.txt', header=None))
    dc_cap_values = np.array(pd.read_csv('~/NetworkDesign/dc20_cust2000_inst1/dc_cap.txt', header=None))
    demand_values = np.array(pd.read_csv('~/NetworkDesign/dc20_cust2000_inst1/demand.txt', header=None))
    
    #Initialize values
    x = utc_values.shape
    numCustomers=x[0]
    numDCs = x[1]
    
    #Create variables
    utc_vars = []    #unit transport cost variables
    dc_cost_vars = []     #distribution centers fixed cost variables
    dc_cap_vars = []
    
    for i in range(1,numCustomers+1):
        
        for j in range(1,numDCs+1):
            v = "x"+str(i)+"_"+str(j)
            v = m.addVar(vtype=GRB.INTEGER, name=v)
            utc_vars.append(v)
            
            if(i==1):
                p = "y"+str(j)
                p = m.addVar(vtype=GRB.BINARY, name=p)
                dc_cost_vars.append(p)
    #Update variables
    m.update()
    
    #Shape arrays to match data shape
    dc_cost_vars = np.array(dc_cost_vars)
    dc_cost_vars = dc_cost_vars.reshape(numDCs,1)
    utc_vars = np.array(utc_vars)
    utc_vars = utc_vars.reshape(numCustomers,numDCs)
    
     
    #Objective Function
    m.setObjective(np.sum(utc_vars*utc_values)+np.sum(dc_cost_vars*dc_cost_values), GRB.MINIMIZE)
    
    #Constraints
    #Customer Demand
    i=-1
    j=-1
    for v in demand_values:
        i+=1
        m.addConstr(np.sum(utc_vars[i])==demand_values[i],"DemandConstr"+str(i+1))
    
    #Distribution Center Capacity Constraints
    for c in dc_cap_values:
        j+=1
        m.addConstr(np.sum(utc_vars[:,j])<=dc_cap_values[j], "CapacityConstr"+str(j+1))
    
    #Optimize model
    m.optimize()
 
    for val in m.getVars():
        if val.x != 0:         #Not to return zero variables
            print val.varName, val.x
        
    print "Objective Value: ", m.objVal
    
    
except GurobiError:
    print 'Error reported. Solution not found'

Optimize a model with 2020 rows, 40020 columns and 80000 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [1e+00, 9e+04]
  Bounds range    [1e+00, 1e+00]
  RHS range       [2e+01, 3e+03]
Found heuristic solution: objective 3.18946e+06
Presolve removed 0 rows and 20 columns
Presolve time: 0.10s
Presolved: 2020 rows, 40000 columns, 80000 nonzeros
Variable types: 0 continuous, 40000 integer (0 binary)

Root relaxation: objective 7.980530e+05, 2602 iterations, 0.09 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    798053.00000 798053.000  0.00%     -    0s

Explored 0 nodes (2602 simplex iterations) in 0.22 seconds
Thread count was 4 (of 4 available processors)

Optimal solution found (tolerance 1.00e-04)
Best objective 7.980530000000e+05, best bound 7.980530000000e+05, gap 0.0%
x1_12 26.0
x2_14 37.0
x3_3 23.0
x3_15

/home/akinfemi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:53: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [3]:
np.sum(utc_vars*utc_values)+np.sum(dc_cost_vars*dc_cost_values)

<gurobi.LinExpr: 87.0 x1_1 + 65.0 x1_2 + 64.0 x1_3 + 35.0 x1_4 + 123.0 x1_5 + 45.0 x1_6 + 89.0 x1_7 + 57.0 x1_8 + 77.0 x1_9 + 83.0 x1_10 + 103.0 x1_11 + 48.0 x1_12 + 90.0 x1_13 + 117.0 x1_14 + 66.0 x1_15 + 99.0 x1_16 + 87.0 x1_17 + 109.0 x1_18 + 34.0 x1_19 + 76.0 x1_20 + 52.0 x2_1 + 65.0 x2_2 + 39.0 x2_3 + 70.0 x2_4 + 31.0 x2_5 + 72.0 x2_6 + 72.0 x2_7 + 50.0 x2_8 + 62.0 x2_9 + 84.0 x2_10 + 23.0 x2_11 + 82.0 x2_12 + 12.0 x2_13 + 14.0 x2_14 + 36.0 x2_15 + 14.0 x2_16 + 16.0 x2_17 + 35.0 x2_18 + 75.0 x2_19 + 54.0 x2_20 + 60.0 x3_1 + 46.0 x3_2 + 34.0 x3_3 + 10.0 x3_4 + 93.0 x3_5 + 30.0 x3_6 + 71.0 x3_7 + 28.0 x3_8 + 54.0 x3_9 + 70.0 x3_10 + 72.0 x3_11 + 39.0 x3_12 + 58.0 x3_13 + 85.0 x3_14 + 35.0 x3_15 + 68.0 x3_16 + 55.0 x3_17 + 81.0 x3_18 + 19.0 x3_19 + 53.0 x3_20 + 91.0 x4_1 + 24.0 x4_2 + 44.0 x4_3 + 47.0 x4_4 + 93.0 x4_5 + 10.0 x4_6 + 45.0 x4_7 + 58.0 x4_8 + 88.0 x4_9 + 39.0 x4_10 + 92.0 x4_11 + 4.0 x4_12 + 73.0 x4_13 + 94.0 x4_14 + 50.0 x4_15 + 73.0 x4_16 + 72.0 x4_17 + 76.0 x4_18 + 17

In [4]:
np.sum(utc_vars[1])==demand_values[1],"DemandConstr"+str(1)

/home/akinfemi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if __name__ == '__main__':


(<gurobi.TempConstr: <gurobi.LinExpr: x2_1 + x2_2 + x2_3 + x2_4 + x2_5 + x2_6 + x2_7 + x2_8 + x2_9 + x2_10 + x2_11 + x2_12 + x2_13 + x2_14 + x2_15 + x2_16 + x2_17 + x2_18 + x2_19 + x2_20> == [37]>,
 'DemandConstr1')

In [24]:
dc_cap_values.shape

(20, 1)

In [8]:
res = DataFrame(columns=["Variable", "Values"])

In [11]:
res.loc[0] = ["z",1]

In [24]:
res.append(DataFrame([['y',2]],columns=['Variable','Values']),ignore_index=True)

,Variable,Values
0,z,1
1,y,2


In [21]:
DataFrame([["y",2]])

,0,1
0,y,2
